In [1]:
%pip install nltk
%pip install pandas
%pip install scikit-learn
%pip install tensorflow
%pip install keras


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from string import punctuation
from nltk import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
from keras.models import Sequential
from keras import layers
from keras.backend import clear_session
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\viswa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\viswa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\viswa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\viswa\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
train_df = pd.read_csv('Dataset/train.csv')

In [4]:
train_df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [5]:
# %%capture
def data_cleanup(train_df):
    train_df['text'] = train_df['text'].str.lower()
    train_df['text'] = train_df['text'].str.strip()
    train_df['text'] = train_df['text'].replace(to_replace ='http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', value = '', regex = True)
    train_df['text'] = train_df['text'].replace(to_replace ='\?*', value = '', regex = True)
    train_df['text'] = train_df['text'].replace(to_replace ='(RT|rt)', value = '', regex = True)
    train_df['text'] = train_df['text'].replace(to_replace ='@[a-z,_]*', value = '', regex = True)
    train_df['text'] = train_df['text'].replace(to_replace ='([0-9]*:[0-9]*)', value = '', regex = True)
    train_df['text'] = train_df['text'].replace(to_replace ='([0-9]*\.[0-9]*)', value = '', regex = True)
    train_df['text'] = train_df['text'].replace(to_replace ='(utc|gmt)', value = '', regex = True)
    train_df['text'] = train_df['text'].replace(to_replace ='_[\S]', value = '', regex = True)
    train_df['text'] = train_df['text'].replace(to_replace ='&amp;?', value = 'and', regex = False)
    train_df['text'] = train_df['text'].replace(to_replace ='&lt;', value = '<', regex = False)
    train_df['text'] = train_df['text'].replace(to_replace ='&gt;', value = '>', regex = False)
    train_df['text'] = train_df['text'].replace(to_replace ='[ ]{2, }', value = ' ', regex = True)
    train_df['text'] = train_df['text'].replace(to_replace ='([^\w\d ]+)', value = '', regex = True)
    return train_df['text']

In [6]:
# %%capture
train_df['text'] = data_cleanup(train_df)

In [7]:
#Split training dataset
tweet_texts = train_df['text']
class_labels = train_df['target']
train_tweets, test_tweets, train_labels, test_labels = train_test_split(tweet_texts,class_labels,test_size=0.2, random_state=42, stratify=class_labels)
train_tweets, validation_tweets, train_labels, validation_labels = train_test_split(train_tweets, train_labels, test_size = 0.2, random_state = 42, stratify=train_labels)

In [9]:
#Text Preprocessor
def preprocessing(text):
   word_lemma = []
   tweet_tokenize = TweetTokenizer()
   tokens = tweet_tokenize.tokenize((text).lower())
   tokens = [w for w in tokens if w not in punctuation and not w.isdigit() and not len(w) < 3]
   stop_words = stopwords.words ('english')
   tweet_without_stopwords = [t for t in tokens if t not in stop_words]
   text = " ".join (tweet_without_stopwords)
   word_lemma = [WordNetLemmatizer().lemmatize(t) for t in tweet_tokenize.tokenize(text)]
   pp_text = " ".join (word_lemma)
   return pp_text

In [10]:
def get_performance_score(actual_label : list, predicted_label : list):
    '''Function to calculate the performance metric using sklearn.
    
    Parameters
    ----------
    actual_label : list
      Actual(Ground Truth) class label from the dataset.
    predicted_label : pd.DataFrame
      Class label predicted by the model
    
    Return
    ------
    f1_score : float
    accuracy : float
    precision : float
    recall : float
    AUROC : float
    '''
    precision = metrics.precision_score(actual_label, predicted_label, pos_label=1)
    recall = metrics.recall_score(actual_label, predicted_label,pos_label=1)
    AUROC = metrics.roc_auc_score(actual_label, predicted_label)
    accuracy = metrics.accuracy_score(actual_label, predicted_label)
    f1_score = metrics.f1_score(actual_label, predicted_label,pos_label=1)
    confusion_mat = metrics.confusion_matrix(actual_label, predicted_label)
    metrics_list = [f1_score, accuracy, precision, recall, AUROC]
    metrics_list = pd.DataFrame(metrics_list).T
    metrics_df = metrics_list.rename(columns={0:'F1',1:'Accuracy',2:'Precision',3:'Recall',4:'AUROC'})
    return metrics_df, confusion_mat

In [11]:
def plot_confusion_matrix(confusion_mat, model_name):
    cm_matplot, cm_ax = plt.subplots(facecolor='#212936')
    # cm_ax.text(color='white')
    cm_plot = ConfusionMatrixDisplay(confusion_matrix=confusion_mat,display_labels=['not_disaster','disaster'])
    title = model_name + " Confusion Matrix"
    cm_plot.plot(cmap=plt.cm.Greens, ax=cm_ax)
    plt.title(title)
    plt.show()

In [12]:
def consolidate_perf_score(models_list, perf_score_list):
  '''Function to consolidate the performance metrics of all the models(KNeighborsClassifier, RandomForestClassifier, LogisticRegression, MLPClassifier) 
  and return a pd.DataFrame.

    Parameters
    ----------
    models_list : list
      List of models.
    perf_score_list : list
      List of performance metrics data frame from various models.
      
    Return
    ------
    consolidated_metrics_df : pd.DataFrame
    '''
  
  consolidated_perf_score_df = pd.concat(perf_score_list)
  consolidated_perf_score_df = consolidated_perf_score_df.rename(columns={0:'F1',1:'Accuracy',2:'Precision',3:'Recall'})
  consolidated_perf_score_df.insert(0,'Model',models_list)
  return consolidated_perf_score_df

# Preprocessing for NN

In [13]:
train_tweets=train_tweets.to_frame()
validation_tweets = validation_tweets.to_frame()
test_tweets = test_tweets.to_frame()

In [14]:
train_tweets['processed_text'] = train_tweets['text'].apply(preprocessing)
validation_tweets['processed_text'] = validation_tweets['text'].apply(preprocessing)
test_tweets['processed_text'] = test_tweets['text'].apply(preprocessing)

In [15]:
tokenizer = Tokenizer(num_words=18000)
tokenizer.fit_on_texts(train_tweets['processed_text'])

X_train = tokenizer.texts_to_sequences(train_tweets['processed_text'])
X_val = tokenizer.texts_to_sequences(validation_tweets['processed_text'])
X_test = tokenizer.texts_to_sequences(test_tweets['processed_text'])

vocab_size = len(tokenizer.word_index) + 1
# print(X_train)

In [16]:


maxlen = 128

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [17]:
clear_session()

# RNN

In [18]:
embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
# model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.add(layers.Dense(10, activation='tanh'))
model.add(layers.Dense(10, activation='gelu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 128, 50)           536250    
                                                                 
 global_max_pooling1d (Glob  (None, 50)                0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 10)                510       
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 10)                110       
                                                        

In [19]:
rnn_model = model
rnn_model.fit(X_train, train_labels,
                    epochs=2,
                    verbose=False,
                    validation_data=(X_val, validation_labels),
                    batch_size=10)

In [20]:
import numpy as np

pred_test = rnn_model.predict(X_test)
# pred_test

48/48 [==============================] - 0s 1ms/step


In [21]:
pred_test = np.where(pred_test > 0.7, 1, 0)
pred_test = pred_test.flatten()

In [23]:
metrics_df_rnn, confusion_mat_rnn = get_performance_score(test_labels, pred_test)

# CNN

In [25]:
inputs = keras.Input(shape=(None,), dtype="int64")
cnn_layer = layers.Embedding(20000, embedding_dim)(inputs)
cnn_layer = layers.Dropout(0.5)(cnn_layer)


cnn_layer = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(cnn_layer)
# cnn_layer = layers.Conv1D(128, 7, padding="valid", activation="softmax", strides=3)(cnn_layer)
# cnn_layer = layers.Conv1D(128, 7, padding="valid", activation="softmax", strides=3)(cnn_layer)
cnn_layer = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(cnn_layer)
cnn_layer = layers.GlobalMaxPooling1D()(cnn_layer)


cnn_layer = layers.Dense(128, activation="relu")(cnn_layer)
cnn_layer = layers.Dropout(0.5)(cnn_layer)

predictions = layers.Dense(1, activation="sigmoid", name="predictions")(cnn_layer)

cnn_model = keras.Model(inputs, predictions)

cnn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
cnn_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 50)          1000000   
                                                                 
 dropout_1 (Dropout)         (None, None, 50)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, None, 128)         44928     
                                                                 
 conv1d_3 (Conv1D)           (None, None, 128)         114816    
                                                                 
 global_max_pooling1d_2 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                             

In [26]:
cnn_model.fit(X_train, train_labels,
                    epochs=2,
                    verbose=False,
                    validation_data=(X_val, validation_labels),
                    batch_size=10)

In [27]:
pred_test = cnn_model.predict(X_test)

pred_test = np.where(pred_test > 0.7, 1, 0)
pred_test = pred_test.flatten()
metrics_df_cnn, confusion_mat_cnn = get_performance_score(test_labels, pred_test)

48/48 [==============================] - 0s 3ms/step


In [28]:
consolidate_perf_score(['RNN', 'CNN'],[metrics_df_rnn, metrics_df_cnn])

,Model,F1,Accuracy,Precision,Recall,AUROC
0,RNN,0.720370,0.801707,0.913146,0.594801,0.776112
0,CNN,0.733096,0.803020,0.876596,0.629969,0.781613
